# Assignment 2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import copy
import os
import sys
np.set_printoptions(threshold=sys.maxsize)
from collections import defaultdict

### Point Cloud Registration

In [2]:
from utils import readData, readPointCloud
from funcs import *

In [3]:
def kitti_velodyne_pcl_registration(path_to_pcl, poses, maxFrame, cam_R):
    
    DownSample = 0.05

    for i in range(maxFrame):
        
        pcl = numpy2open3d_pcl(readPointCloud(os.path.join(path_to_pcl,str(i).zfill(6) + '.bin'))).voxel_down_sample(voxel_size = DownSample)
        P_mat = np.reshape(poses[i],(3,4))
        
        pcl = transform_pcl_to_new_frame(pcl, cam_R)
        R,t = get_Rt_from_3_4_projection_matrix(P_mat)
        pcl = rotate(pcl, R)
        pcl = translate(pcl, t)
        pcl = transform_pcl_to_new_frame(pcl, cam_R.T)
        
        if(i == 0): final_pcl  = pcl
        else:
            final_pcl = final_pcl + pcl

    return final_pcl

In [4]:
path_to_pcl =  './dataset/01/'
path_to_dataset = './dataset/'
poses = readData(path_to_dataset+'01.txt')
cam_R = angles2rotmat(-np.pi/2,np.pi/2,0)
pcl = kitti_velodyne_pcl_registration(path_to_pcl, poses, 77, cam_R)

In [5]:
custom_draw_geometry(pcl, save_name = './Point_Cloud_2_1.jpg')

## Occupancy Map Construction

In [6]:
def open3d2numpy(pcl):
    return np.asarray(pcl.points)
    
def genOccupancy(path_to_pcl, poses, cam_R, resolution = 5, maxFrame = 77, threshold = 2, threshold_above_road = 0, start = 0, fl = False):
    DownSample = 0.05
    unique_z_locs = []
    for i in range(start,maxFrame,int(maxFrame/resolution)):
        pcl = numpy2open3d_pcl(readPointCloud(os.path.join(path_to_pcl,str(i).zfill(6) + '.bin'))).voxel_down_sample(voxel_size = DownSample)
        P_mat = np.reshape(poses[i],(3,4))
        
        pcl = transform_pcl_to_new_frame(pcl, cam_R)
        R,t = get_Rt_from_3_4_projection_matrix(P_mat)
        pcl = rotate(pcl, R)
        pcl = translate(pcl, t)
        pcl = transform_pcl_to_new_frame(pcl, cam_R.T)
        if(i == 0 or fl): final_pcl  = pcl
        else:
            final_pcl = final_pcl + pcl

    points = np.around(np.asarray(final_pcl.points),2)

    points_dict = defaultdict(int)
    
    for point in points:
        if point[2]>threshold_above_road:
            points_dict[tuple(point[:2])] += 1
    
    keys = [key for key in points_dict.keys()]
    for key in keys:
        if(points_dict[key] < threshold):
            _ = points_dict.pop(key,None)
    
    points = [np.array([key[0],key[1],0]) for key in points_dict.keys()]
    final_pcl.points = o3d.utility.Vector3dVector(points)
    final_pcl.paint_uniform_color(np.array([1,1,1]))
    return final_pcl

In [7]:
occ_gen_5 = genOccupancy(path_to_pcl, poses, cam_R, 5)
occ_gen_10 = genOccupancy(path_to_pcl, poses, cam_R, 10)
occ_gen_15 = genOccupancy(path_to_pcl, poses, cam_R, 15)

In [10]:
custom_draw_geometry_with_key_callback(occ_gen_5, './2_2_Occ_Map_05_Frames.jpg')

In [11]:
custom_draw_geometry_with_key_callback(occ_gen_10, './2_2_Occ_Map_10_Frames.jpg')

In [12]:
custom_draw_geometry_with_key_callback(occ_gen_15, './2_2_Occ_Map_15_Frames.jpg')

In [17]:
def save_lidar_occupancy_scans(path_to_pcl, poses, camR, path_to_results, maxFrame, threshold = 2):
    try:
        os.makedirs(path_to_results)
    except OSError:
        print('[LOG]: Result Directory already exists.')
    for i in range(maxFrame):
        pcl = genOccupancy(path_to_pcl, poses, camR, resolution = i+1, maxFrame = i+1, threshold = threshold, start = i, fl = True)
        custom_draw_geometry_with_key_callback(pcl, path_to_results+'occ_scan_'+str(i).zfill(2)+'.jpg')
        print('[LOG]: Saved file at '+path_to_results+'occ_scan_'+str(i).zfill(2)+'.jpg')

In [18]:
save_lidar_occupancy_scans(path_to_pcl, poses, cam_R, './Results/', 77, threshold = 2)

[LOG]: Result Directory already exists.
[LOG]: Saved file at ./Results/occ_scan_00.jpg
[LOG]: Saved file at ./Results/occ_scan_01.jpg
[LOG]: Saved file at ./Results/occ_scan_02.jpg
[LOG]: Saved file at ./Results/occ_scan_03.jpg
[LOG]: Saved file at ./Results/occ_scan_04.jpg
[LOG]: Saved file at ./Results/occ_scan_05.jpg
[LOG]: Saved file at ./Results/occ_scan_06.jpg
[LOG]: Saved file at ./Results/occ_scan_07.jpg
[LOG]: Saved file at ./Results/occ_scan_08.jpg
[LOG]: Saved file at ./Results/occ_scan_09.jpg
[LOG]: Saved file at ./Results/occ_scan_10.jpg
[LOG]: Saved file at ./Results/occ_scan_11.jpg
[LOG]: Saved file at ./Results/occ_scan_12.jpg
[LOG]: Saved file at ./Results/occ_scan_13.jpg
[LOG]: Saved file at ./Results/occ_scan_14.jpg
[LOG]: Saved file at ./Results/occ_scan_15.jpg
[LOG]: Saved file at ./Results/occ_scan_16.jpg
[LOG]: Saved file at ./Results/occ_scan_17.jpg
[LOG]: Saved file at ./Results/occ_scan_18.jpg
[LOG]: Saved file at ./Results/occ_scan_19.jpg
[LOG]: Saved file at